In [ ]:
import numpy as np
import json
import matplotlib.pyplot as plt
import keras_ocr

In [ ]:
# Load the pre-trained model
pipeline = keras_ocr.pipeline.Pipeline() 

In [ ]:
 # Provide an image for OCR
image_path = './02.png'
image = keras_ocr.tools.read(image_path)


In [ ]:
# Perform OCR
#predictions = pipeline.recognize([image])
prediction_groups = pipeline.recognize([image]) 
print(prediction_groups)

In [ ]:
keras_ocr.tools.drawAnnotations(image=image, predictions=prediction_groups[0])
 

In [34]:
 #word, ( [top-left], (top-right), [bottom-right], bottom-left)
 
 prediction_groups[0][0]

('o',
 array([[558.8086 , 292.38672],
        [612.9961 , 292.38672],
        [612.9961 , 366.89453],
        [558.8086 , 366.89453]], dtype=float32))

In [35]:
# raw_detections =[]
# for predictions in prediction_groups[0]:
#     raw_detections.append(predictions[0])

# print(f'Detections:{raw_detections') 

SyntaxError: f-string: expecting '}' (2816776938.py, line 5)